In [1]:
import sys
import warnings
warnings.filterwarnings('ignore')
sys.path.append('C:/Users/yixin/Desktop/Machine_Learning_Projects/restaurant-visitor-forecasting')

import helper
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from geopy.distance import great_circle
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn import metrics, model_selection

In [2]:
# Define variables
seed = 2018
path = 'C:/Users/yixin/Desktop/Machine_Learning_Projects/restaurant-visitor-forecasting'
# features that will be one-hot encoded
cat = ['day_of_week', 'weekend_holiday', 'air_genre_name', 'prefecture']
# features that will be target encoded
agg = ['day_of_week', 'year', 'weekend_holiday',
       'air_store_id']
# Some engineered features are unhelpful, and will be dropped
drop = ['visit_date', 'visitors', 'air_area_name', 'month', 'weight',
        'count_reserve_1', 'count_reserve_2', 'count_reserve_3', 'count_reserve_7', 
        'count_reserve_14', 'count_reserve_21', 'count_reserve_28', 'count_reserve_35']

In [3]:
# Read data
data = {
    'air_reserve': pd.read_csv(path + '/input/air_reserve.csv', \
                               parse_dates=['visit_datetime', 'reserve_datetime']),
    'hpg_reserve': pd.read_csv(path + '/input/hpg_reserve.csv', \
                               parse_dates=['visit_datetime', 'reserve_datetime']),
    'air_visit': pd.read_csv(path + '/input/air_visit_data.csv', parse_dates=['visit_date']), # training set
    'holidays': pd.read_csv(path + '/input/date_info.csv', parse_dates=['calendar_date']).rename(
        columns={'calendar_date': 'visit_date'}),
    'air_store': pd.read_csv(path + '/input/air_store_info.csv'),
    'id': pd.read_csv(path + '/input/store_id_relation.csv'),
    'submission': pd.read_csv(path + '/input/sample_submission.csv'),  # test set
}
train_window = pd.read_csv(path + '/output/train_window.csv', parse_dates=['visit_date'])
test_window = pd.read_csv(path + '/output/test_window.csv', parse_dates=['visit_date'])

data['hpg_reserve'] = pd.merge(data['hpg_reserve'], data['id'],
                               how='inner', on='hpg_store_id').drop('hpg_store_id', axis=1)

In [4]:
#######################################################################################################
###                                      Feature Engineering                                        ###
#######################################################################################################
# Add day of week, month, year, etc into air visit dataset
data['submission']['visit_date'] = data['submission']['id'].apply(lambda x:x[-10:])
data['submission']['visit_date'] = pd.to_datetime(data['submission']['visit_date'])
data['submission']['air_store_id'] = data['submission']['id'].apply(lambda x:x[:-11])
data['submission'].drop('id', axis=1, inplace=True)

for df in ['air_visit', 'submission']:
    data[df]['day_of_week'] = data[df]['visit_date'].dt.dayofweek
    data[df]['month'] = data[df]['visit_date'].dt.month
    data[df]['year'] = data[df]['visit_date'].dt.year

le = LabelEncoder()
train, test = data['air_visit'], data['submission']
train['date_int'] = train['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)   # possible
test['date_int'] = test['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)     # possible
train['year'] = le.fit_transform(train['year'])
test['year'] = le.transform(test['year'])

visit_date = pd.DataFrame(data['air_visit']['visit_date'].unique()).rename(columns={0: 'visit_date'})
visit_date = visit_date.sort_values('visit_date').reset_index(drop=True)
visit_date['weight'] = ((visit_date.index + 1) / visit_date.shape[0]) ** 5
train = train.merge(visit_date, on='visit_date', how='left')
test = test.merge(visit_date, on='visit_date', how='left')

In [5]:
# Aggregate reserve_visitors grouped by each store and visit date
reserve = pd.concat([data['air_reserve'], data['hpg_reserve']])
reserve['visit_date'] = reserve['visit_datetime'].dt.date
reserve['visit_date'] = pd.to_datetime(reserve['visit_date'])

tmp1 = reserve.groupby(['air_store_id', 'visit_date'], as_index=False)[
    'reserve_visitors'].sum().rename(columns={'reserve_visitors': 'sum_reserve'})
tmp2 = reserve.groupby(['air_store_id', 'visit_date'], as_index=False)[
    'reserve_visitors'].mean().rename(columns={'reserve_visitors': 'mean_reserve'})
reserve = pd.merge(tmp1, tmp2, how='inner', on=['air_store_id', 'visit_date'])

# Merge training and test sets with reserve
train = train.merge(reserve, how='left', on=['air_store_id', 'visit_date'])
test = test.merge(reserve, how='left', on=['air_store_id', 'visit_date'])

In [6]:
### Generate rolling time windows data
##train_window = helper.rolling_window(train, reserve, columns=['sum_reserve', 'count_reserve', 'mean_reserve'])
##test_window = helper.rolling_window(test, reserve, columns=['sum_reserve', 'count_reserve', 'mean_reserve'])

# Merge training and test sets with windows
train = train.merge(train_window, how='left', on=['air_store_id', 'visit_date'])
test = test.merge(test_window, how='left', on=['air_store_id', 'visit_date'])

In [7]:
# Add weekend_holiday categories into holidays data
weekday = ['Monday', 'Tuesday', 'Wednesday', 'Thursday']
weekend = ['Friday', 'Saturday', 'Sunday']
data['holidays']['weekend_holiday'] = 'Nonweekend/Nonholiday'
data['holidays']['weekend_holiday'][(data['holidays']['day_of_week'].isin(weekday)) &
                                    (data['holidays']['holiday_flg'] == 1)] = 'Nonweekend/Holiday'
data['holidays']['weekend_holiday'][(data['holidays']['day_of_week'].isin(weekend)) & 
                                    (data['holidays']['holiday_flg'] == 0)] = 'Weekend/Nonholiday'
data['holidays']['weekend_holiday'][(data['holidays']['day_of_week'].isin(weekend)) & 
                                    (data['holidays']['holiday_flg'] == 1)] = 'Weekend/Holiday'
data['holidays'].drop(['day_of_week', 'holiday_flg'], axis=1, inplace=True)

# Merge training and test sets with holidays
train = train.merge(data['holidays'], how='left', on='visit_date')
test = test.merge(data['holidays'], how='left', on='visit_date')

In [8]:
# Add engineered features into air_store
# distance difference (reference is median latitude and median longitude)
ref = data['air_store'][['latitude', 'longitude']].median().values
data['air_store']['diff_dist'] = data['air_store'].apply(lambda x: \
                                great_circle((x['latitude'],x['longitude']), ref).km, axis = 1)
# location difference (reference is median latitude and median longitude)
data['air_store']['diff_lat'] = np.absolute(
    data['air_store']['latitude'].median() - data['air_store']['latitude'])
data['air_store']['diff_long'] = np.absolute(
    data['air_store']['longitude'].median() - data['air_store']['longitude'])
# prefecture
data['air_store']['prefecture'] = data['air_store']['air_area_name'].apply(lambda x:str(x).split(' ')[0])
# number of restaurants per area
tmp = data['air_store'].groupby('air_area_name', as_index=False)['air_store_id'].count().rename(
    columns={'air_store_id': 'rest_per_area'})
data['air_store'] = pd.merge(data['air_store'], tmp, how='left', on='air_area_name')
# number of genre per area
tmp = data['air_store'].groupby('air_area_name')['air_genre_name'].nunique().reset_index().rename(
    columns={'air_genre_name': 'genre_per_area'})
data['air_store'] = pd.merge(data['air_store'], tmp, how='left', on='air_area_name')
# number of restaurants per area per genre
tmp = data['air_store'].groupby(['air_area_name', 'air_genre_name'], as_index=False)[
    'air_store_id'].count().rename(columns={'air_store_id': 'rest_per_area_grouped_by_genre'})
data['air_store'] = pd.merge(data['air_store'], tmp, how='left', on=['air_area_name', 'air_genre_name'])
# air_store_id
tmp = data['air_store'].sort_values(['longitude', 'latitude'])
strId_to_intId = dict(zip(tmp['air_store_id'], [i for i in range(tmp.shape[0])]))
data['air_store']['id'] = data['air_store']['air_store_id'].apply(lambda x:strId_to_intId[x])

# Merge training and test sets with air_store
train = pd.merge(train, data['air_store'], how='left', on='air_store_id')
test = pd.merge(test, data['air_store'], how='left', on='air_store_id')
train = train[train['air_store_id'].isin(test['air_store_id'])]

In [9]:
train = train.fillna(-1)
test = test.fillna(-1)

In [10]:
# Adversarial validation 
train, test = helper.get_isTest(train, test, cat, 
                                [c for c in drop if c!='month' and c!='isTest'], threshold=0.12, seed=seed)
train[train['isTest']==1].shape, train[train['isTest']==1]['air_store_id'].nunique()

AUC on train: 0.9978859199572827, AUC on test: 0.9974262842099478


((11330, 48), 817)

In [11]:
# Define evaluation metric
def RMSE(y_true, y_pred):
    return metrics.mean_squared_error(y_true, y_pred) ** 0.5
rmse = metrics.make_scorer(metrics.mean_squared_error, greater_is_better=False)

In [42]:
#######################################################################################################
###                                           Training                                              ###
#######################################################################################################
reg = XGBRegressor(eta=0.01, learning_rate=0.01, n_estimators=10000, 
                   max_depth=8, min_child_weight=10,
                   gamma=0.8,
                   subsample=0.95, colsample_bytree=0.85,
                   reg_alpha=2, reg_lambda=0,
                   eval_metric='rmse', random_state=seed, seed=seed, n_jobs=-1, missing=-1,)
reg

'''
reg = RandomForestRegressor(n_estimators=700, 
                            max_depth=23, 
                            min_samples_split=21, min_samples_leaf=5,
                            max_features=0.45,
                            random_state=seed, n_jobs=-1)
reg
'''

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=23,
           max_features=0.45, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=5, min_samples_split=21,
           min_weight_fraction_leaf=0.0, n_estimators=700, n_jobs=-1,
           oob_score=False, random_state=2018, verbose=0, warm_start=False)

In [41]:
# Two-level cross-validation for selecting optimal parameters
kf1 = model_selection.KFold(n_splits=5, shuffle=True, random_state=seed+2018030)
kf2 = model_selection.KFold(n_splits=10, shuffle=True, random_state=seed+2018030)

for n_estimators in [800]:
        print('################ n_estimators: {} #################'.format(n_estimators))
        i = 0; rmsle_valid = np.array([0.] * 5)
        for train_idx, valid_idx in kf1.split(train):
            # first-level cross-validation
            train_cv, valid_cv = train.iloc[train_idx], train.iloc[valid_idx]
            X_train_cv, _, y_train_cv, _ = helper.getData(train_cv, valid_cv, seed=seed, ohe=True,
                                                          agg_columns=agg, cat_columns=cat)
            sample_weight = X_train_cv['weight']
            
            # second-level cross-validation, used for grand average of target encoding
            valid_cv = helper.secondLevelCV(kf2, train_cv, valid_cv, agg_columns=agg, seed=seed)
            X_valid_cv, y_valid_cv = helper.feature_encoder(valid_cv, cat)
            X_valid_cv = X_valid_cv[X_train_cv.columns]
            X_train_cv.drop(drop, axis=1, inplace=True)
            X_valid_cv.drop(drop, axis=1, inplace=True)
            
            # impute missing data in validation set
            neigh = KNeighborsRegressor(n_neighbors=5, weights='uniform', metric='euclidean', n_jobs=-1)
            helper.imputer(X_train_cv, X_valid_cv, neigh)
            
            # assign sample weight to samples that have same distribution as test set
            idx_trn, idx_val = X_train_cv['year'] == 1, X_valid_cv['year'] == 1
            
            reg.set_params(n_estimators=n_estimators)
            reg.fit(X_train_cv.drop('air_store_id', axis=1), y_train_cv, sample_weight=sample_weight)
            yhat_train = reg.predict(X_train_cv.drop('air_store_id', axis=1))[idx_trn]
            yhat_valid = reg.predict(X_valid_cv.drop('air_store_id', axis=1))[idx_val]
            print('RMSLE on training set:', RMSE(y_train_cv.loc[idx_trn], yhat_train))
            print('RMSLE on validation set:', RMSE(y_valid_cv.loc[idx_val], yhat_valid))
            print('*************************************')
            rmsle_valid[i] = RMSE(y_valid_cv.loc[idx_val], yhat_valid); i += 1
        print('Average RMSLE on validation set:', rmsle_valid.mean(), rmsle_valid.std())

################ n_estimators: 800 #################
RMSLE on training set: 0.375244769022
RMSLE on validation set: 0.479432157866
*************************************
RMSLE on training set: 0.376162455569
RMSLE on validation set: 0.475774051047
*************************************
RMSLE on training set: 0.374663182096
RMSLE on validation set: 0.479531206343
*************************************
RMSLE on training set: 0.37455917417
RMSLE on validation set: 0.480181070178
*************************************
RMSLE on training set: 0.374510755675
RMSLE on validation set: 0.48255182364
*************************************
Average RMSLE on validation set: 0.479494061815 0.00217557255898


In [43]:
# Train on entire training set using optimal parameters and predict on test set
'''kf2 = model_selection.KFold(n_splits=10, shuffle=True, random_state=seed+20180106)

X_train, _, y_train, _ = helper.getData(train, test, seed=seed, ohe=True,
                                            agg_columns=agg, cat_columns=cat)
sample_weight = X_train['weight']
test = helper.secondLevelCV(kf2, train, test, agg_columns=agg, seed=seed)
X_test, y_test = helper.feature_encoder(test, cat)
X_test = X_test[X_train.columns]
X_train.drop(drop, axis=1, inplace=True)
X_test.drop(drop, axis=1, inplace=True)'''

X_train, X_test, y_train, _ = helper.getData(train, test, seed=seed, ohe=True,
                                            agg_columns=agg, cat_columns=cat)
sample_weight = X_train['weight']
X_train.drop(drop, axis=1, inplace=True)
X_test.drop(drop, axis=1, inplace=True)

neigh = KNeighborsRegressor(n_neighbors=5, weights='uniform', metric='euclidean', n_jobs=-1)
helper.imputer(X_train, X_test, neigh)

idx_trn = X_train['year'] == 1
X_train['isTest'] = X_train['isTest'].astype(np.int8)
X_test['isTest'] = X_test['isTest'].astype(np.int8)
reg.fit(X_train.drop('air_store_id', axis=1), y_train, sample_weight=sample_weight)
yhat_train = reg.predict(X_train.drop('air_store_id', axis=1))[idx_trn]
yhat_test = reg.predict(X_test.drop('air_store_id', axis=1))
print('RMSLE on entire training set:', RMSE(y_train.loc[idx_trn], yhat_train))

RMSLE on entire training set: 0.37860233098


In [44]:
# Generate submission file
sub = pd.concat([X_test.reset_index(drop=True), 
                 pd.DataFrame(yhat_test).rename(columns={0: 'visitors'})], axis=1).drop([
    c for c in X_test.columns if c not in ['air_store_id', 'date_int', 'visitors']], axis=1)
sub['date_int'] = sub['date_int'].astype(str).apply(lambda x:x[:4] + '-' + x[4:6] + '-' + x[6:])
sub['id'] = sub['air_store_id'] + '_' + sub['date_int']
sub.drop(['air_store_id', 'date_int'], axis=1, inplace=True)

submission = pd.read_csv(path + '/input/sample_submission.csv').drop('visitors', axis=1)
submission = submission.merge(sub, on='id', how='left')
submission['visitors'] = np.expm1(submission['visitors']).clip(lower=0.)

In [46]:
submission.to_csv(path + '/output/train1_rf_20180125.csv', index=False)

In [52]:
# Save model
with open(path + '/output/train1_rf_20180125.data', 'wb') as f:
    pickle.dump(reg, f)
with open(path + '/output/columns_ordering.data', 'wb') as f:
    pickle.dump(X_train_cv.columns, f)